In [1]:
import pandas as pd

# data

In [2]:
data_path = "../data/"

In [1]:
df_orig_filename = "article corpus.xlsx"

In [4]:
df_orig = pd.read_excel(data_path + df_orig_filename)
df_orig.columns = [label.strip() for label in df_orig.columns]
df_orig.columns

Index(['Title', 'Source', 'Data', 'Link', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [5]:
select_columns = ["Source","Data","Link"]
rename_columns = ["source","date","url"]
df_orig = df_orig[select_columns]
df_orig.columns = rename_columns
df_orig.head()

,source,date,url
0,Хабар,5.01.2022,https://khabar.kz/ru/news/obshchestvo/item/137...
1,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137784...
2,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137785...
3,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137786...
4,Хабар,5.01.2022,https://khabar.kz/ru/news/obshchestvo/item/137...


In [7]:
df_orig.shape

(510, 3)

In [34]:
df_orig = df_orig.drop_duplicates()
df_orig.shape

(495, 6)

# analysis

In [35]:
path_analysis = "../analysis/"

In [36]:
df_counts_filename = "article_kcounts_cluster_graph.xlsx"

In [37]:
df_counts = pd.read_excel(path_analysis + df_counts_filename,sheet_name="Sheet1")

In [38]:
df_counts.shape

(495, 10)

In [39]:
df_counts.head()

,source,date,url,year,month,day,cluster,keyword1_count,keyword2_count,_merge
0,nur.kz,5.01.2022,https://www.nur.kz/politics/kazakhstan/1949858...,2022,1,5,1,12,0,both
1,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137805...,2022,1,5,1,9,0,both
2,Хабар,11.01.2022,https://khabar.kz/ru/news/politika/item/138023...,2022,1,11,1,8,0,both
3,nur.kz,10.01.2022,https://www.nur.kz/politics/kazakhstan/1950256...,2022,1,10,1,6,0,both
4,akorda.kz,11.01.2022,https://www.akorda.kz/ru/vystuplenie-glavy-gos...,2022,1,11,1,6,0,both


In [40]:
df_counts = df_counts.drop("_merge",axis=1)
df_counts.shape

(495, 9)

# Step 0. Format Date

In [41]:
df_orig["year"] = df_orig.loc[:,"date"].apply(lambda x: int(x.split(".")[2]))

In [42]:
df_orig["month"] = df_orig.loc[:,"date"].apply(lambda x: int(x.split(".")[1]))

In [43]:
df_orig["day"] = df_orig.loc[:,"date"].apply(lambda x: int(x.split(".")[0]))

In [44]:
df_orig.head()

,source,date,url,year,month,day
0,Хабар,5.01.2022,https://khabar.kz/ru/news/obshchestvo/item/137...,2022,1,5
1,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137784...,2022,1,5
2,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137785...,2022,1,5
3,Хабар,5.01.2022,https://khabar.kz/ru/news/politika/item/137786...,2022,1,5
4,Хабар,5.01.2022,https://khabar.kz/ru/news/obshchestvo/item/137...,2022,1,5


# Step 1. Merge df_orig & df_count

In [49]:
pk_columns = ["url"]
df_merge = df_orig[pk_columns].merge(df_counts,on="url",how="inner",indicator=True)

In [50]:
df_merge.shape

(495, 10)

In [51]:
df_merge.groupby("_merge").count()

,url,source,date,year,month,day,cluster,keyword1_count,keyword2_count
_merge,,,,,,,,,
left_only,0,0,0,0,0,0,0,0,0
right_only,0,0,0,0,0,0,0,0,0
both,495,495,495,495,495,495,495,495,495


In [52]:
df_merge.head()

,url,source,date,year,month,day,cluster,keyword1_count,keyword2_count,_merge
0,https://khabar.kz/ru/news/obshchestvo/item/137...,Хабар,5.01.2022,2022,1,5,1,0,2,both
1,https://khabar.kz/ru/news/politika/item/137784...,Хабар,5.01.2022,2022,1,5,1,4,0,both
2,https://khabar.kz/ru/news/politika/item/137785...,Хабар,5.01.2022,2022,1,5,1,0,0,both
3,https://khabar.kz/ru/news/politika/item/137786...,Хабар,5.01.2022,2022,1,5,1,2,4,both
4,https://khabar.kz/ru/news/obshchestvo/item/137...,Хабар,5.01.2022,2022,1,5,1,0,0,both


# Step 2. create time series 

In [53]:
df_merge.sort_values(["year","month","day"]).head()

,url,source,date,year,month,day,cluster,keyword1_count,keyword2_count,_merge
73,https://www.zakon.kz/6003471-deputaty-mazhilis...,zakon.kz,3.01.2022,2022,1,3,2,8,0,both
183,https://www.nur.kz/politics/kazakhstan/1949685...,nur.kz,3.01.2022,2022,1,3,1,2,0,both
184,https://www.nur.kz/politics/kazakhstan/1949639...,nur.kz,3.01.2022,2022,1,3,1,2,0,both
210,https://ru.sputnik.kz/20220103/tokaev-dal-sroc...,sputnik.kz,3.01.2022,2022,1,3,2,0,0,both
211,https://ru.sputnik.kz/20220103/Chto-proiskhodi...,sputnik.kz,3.01.2022,2022,1,3,2,0,10,both


In [54]:
ts_dat = df_merge.groupby(["year","month","cluster"],as_index=False).agg({"url":"count","keyword1_count":"sum","keyword2_count":"sum"})
ts_dat.head()

,year,month,cluster,url,keyword1_count,keyword2_count
0,2022,1,1,99,147,14
1,2022,1,2,191,163,146
2,2022,2,1,28,70,6
3,2022,2,2,36,44,4
4,2022,3,1,11,29,0


In [55]:
ts_dat["k1_freq"] = ts_dat["keyword1_count"] / ts_dat["url"]
ts_dat["k2_freq"] = ts_dat["keyword2_count"] / ts_dat["url"]
ts_dat.head()

,year,month,cluster,url,keyword1_count,keyword2_count,k1_freq,k2_freq
0,2022,1,1,99,147,14,1.484848,0.141414
1,2022,1,2,191,163,146,0.853403,0.764398
2,2022,2,1,28,70,6,2.500000,0.214286
3,2022,2,2,36,44,4,1.222222,0.111111
4,2022,3,1,11,29,0,2.636364,0.000000


In [56]:
months = pd.DataFrame(data={"month":range(1,13)})
years = pd.DataFrame(data={"year":range(2022,2024)})
cluster = pd.DataFrame(data={"cluster":range(1,3)})
ts = years.merge(months,how="cross")
ts = ts.merge(cluster,how="cross")
ts

,year,month,cluster
0,2022,1,1
1,2022,1,2
2,2022,2,1
3,2022,2,2
4,2022,3,1
5,2022,3,2
6,2022,4,1
7,2022,4,2
8,2022,5,1
9,2022,5,2


In [57]:
ts = ts.merge(ts_dat,on=["year","month","cluster"],how="left")

In [58]:
ts

,year,month,cluster,url,keyword1_count,keyword2_count,k1_freq,k2_freq
0,2022,1,1,99.0,147.0,14.0,1.484848,0.141414
1,2022,1,2,191.0,163.0,146.0,0.853403,0.764398
2,2022,2,1,28.0,70.0,6.0,2.500000,0.214286
3,2022,2,2,36.0,44.0,4.0,1.222222,0.111111
4,2022,3,1,11.0,29.0,0.0,2.636364,0.000000
5,2022,3,2,19.0,30.0,7.0,1.578947,0.368421
6,2022,4,1,7.0,20.0,2.0,2.857143,0.285714
7,2022,4,2,5.0,0.0,0.0,0.000000,0.000000
8,2022,5,1,3.0,6.0,0.0,2.000000,0.000000
9,2022,5,2,2.0,0.0,0.0,0.000000,0.000000


In [59]:
ts = ts[~((ts["year"]==2023) & (ts["month"]>5))]
ts

,year,month,cluster,url,keyword1_count,keyword2_count,k1_freq,k2_freq
0,2022,1,1,99.0,147.0,14.0,1.484848,0.141414
1,2022,1,2,191.0,163.0,146.0,0.853403,0.764398
2,2022,2,1,28.0,70.0,6.0,2.500000,0.214286
3,2022,2,2,36.0,44.0,4.0,1.222222,0.111111
4,2022,3,1,11.0,29.0,0.0,2.636364,0.000000
5,2022,3,2,19.0,30.0,7.0,1.578947,0.368421
6,2022,4,1,7.0,20.0,2.0,2.857143,0.285714
7,2022,4,2,5.0,0.0,0.0,0.000000,0.000000
8,2022,5,1,3.0,6.0,0.0,2.000000,0.000000
9,2022,5,2,2.0,0.0,0.0,0.000000,0.000000


In [71]:
ts["yearmonth"] = ts.apply(lambda x: str(x["year"])+"."+str(x["month"]),axis=1)
ts

C:\Users\seoul\AppData\Local\Temp\ipykernel_21716\680612533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts["yearmonth"] = ts.apply(lambda x: str(x["year"])+"."+str(x["month"]),axis=1)


,year,month,cluster,url,keyword1_count,keyword2_count,k1_freq,k2_freq,yearmonth
0,2022,1,1,99.0,147.0,14.0,1.484848,0.141414,2022.1
1,2022,1,2,191.0,163.0,146.0,0.853403,0.764398,2022.1
2,2022,2,1,28.0,70.0,6.0,2.500000,0.214286,2022.2
3,2022,2,2,36.0,44.0,4.0,1.222222,0.111111,2022.2
4,2022,3,1,11.0,29.0,0.0,2.636364,0.000000,2022.3
5,2022,3,2,19.0,30.0,7.0,1.578947,0.368421,2022.3
6,2022,4,1,7.0,20.0,2.0,2.857143,0.285714,2022.4
7,2022,4,2,5.0,0.0,0.0,0.000000,0.000000,2022.4
8,2022,5,1,3.0,6.0,0.0,2.000000,0.000000,2022.5
9,2022,5,2,2.0,0.0,0.0,0.000000,0.000000,2022.5


In [72]:
ts.loc[ts["cluster"].isna(),"cluster"] = 0
ts.loc[ts["url"].isna(),"url"] = 0
ts.loc[ts["keyword1_count"].isna(),"keyword1_count"] = 0
ts.loc[ts["keyword2_count"].isna(),"keyword2_count"] = 0
ts.loc[ts["k1_freq"].isna(),"k1_freq"] = 0
ts.loc[ts["k2_freq"].isna(),"k2_freq"] = 0
ts

,year,month,cluster,url,keyword1_count,keyword2_count,k1_freq,k2_freq,yearmonth
0,2022,1,1,99.0,147.0,14.0,1.484848,0.141414,2022.1
1,2022,1,2,191.0,163.0,146.0,0.853403,0.764398,2022.1
2,2022,2,1,28.0,70.0,6.0,2.500000,0.214286,2022.2
3,2022,2,2,36.0,44.0,4.0,1.222222,0.111111,2022.2
4,2022,3,1,11.0,29.0,0.0,2.636364,0.000000,2022.3
5,2022,3,2,19.0,30.0,7.0,1.578947,0.368421,2022.3
6,2022,4,1,7.0,20.0,2.0,2.857143,0.285714,2022.4
7,2022,4,2,5.0,0.0,0.0,0.000000,0.000000,2022.4
8,2022,5,1,3.0,6.0,0.0,2.000000,0.000000,2022.5
9,2022,5,2,2.0,0.0,0.0,0.000000,0.000000,2022.5


In [73]:
ts.loc[:,"cluster"] = ts["cluster"].astype(int)
ts.loc[:,"url"] = ts["url"].astype(int)
ts.loc[:,"keyword1_count"] = ts["keyword1_count"].astype(int)
ts.loc[:,"keyword2_count"] = ts["keyword2_count"].astype(int)
ts

,year,month,cluster,url,keyword1_count,keyword2_count,k1_freq,k2_freq,yearmonth
0,2022,1,1,99.0,147.0,14.0,1.484848,0.141414,2022.1
1,2022,1,2,191.0,163.0,146.0,0.853403,0.764398,2022.1
2,2022,2,1,28.0,70.0,6.0,2.500000,0.214286,2022.2
3,2022,2,2,36.0,44.0,4.0,1.222222,0.111111,2022.2
4,2022,3,1,11.0,29.0,0.0,2.636364,0.000000,2022.3
5,2022,3,2,19.0,30.0,7.0,1.578947,0.368421,2022.3
6,2022,4,1,7.0,20.0,2.0,2.857143,0.285714,2022.4
7,2022,4,2,5.0,0.0,0.0,0.000000,0.000000,2022.4
8,2022,5,1,3.0,6.0,0.0,2.000000,0.000000,2022.5
9,2022,5,2,2.0,0.0,0.0,0.000000,0.000000,2022.5


In [74]:
index_columns = ["year","month","yearmonth"]
pivot_columns = ["cluster"]
value_columns = ["url","k1_freq","k2_freq"]
ts_pivot = ts[index_columns+pivot_columns+value_columns]
ts_pivot = ts.pivot(index = index_columns,columns = pivot_columns,values = value_columns)
ts_pivot

url           k1_freq             k2_freq          
cluster                  1      2          1         2         1         2
year month yearmonth                                                      
2022 1     2022.1     99.0  191.0   1.484848  0.853403  0.141414  0.764398
     2     2022.2     28.0   36.0   2.500000  1.222222  0.214286  0.111111
     3     2022.3     11.0   19.0   2.636364  1.578947  0.000000  0.368421
     4     2022.4      7.0    5.0   2.857143  0.000000  0.285714  0.000000
     5     2022.5      3.0    2.0   2.000000  0.000000  0.000000  0.000000
     6     2022.6     14.0    8.0   3.500000  0.750000  0.214286  0.000000
     7     2022.7      3.0    3.0   2.333333  2.000000  0.000000  0.000000
     8     2022.8      0.0    7.0   0.000000  1.428571  0.000000  0.000000
     9     2022.9      7.0    1.0   3.571429  0.000000  0.000000  0.000000
     10    2022.10     4.0    4.0   4.750000  0.000000  0.000000  0.000000
     11    2022.11     4.0    3.0   3.750000  2.333333  0.000000  0.000000
     12    2022.12     4.0    8.0   1.500000  2.250000  0.000000  0.125000
2023 1     2023.1      4.0   17.0   1.750000  1.470588  0.000000  0.705882
     2     2023.2      0.0    0.0   0.000000  0.000000  0.000000  0.000000
     3     2023.3      1.0    0.0  10.000000  0.000000  0.000000  0.000000
     4     2023.4      0.0    0.0   0.000000  0.000000  0.000000  0.000000
     5     2023.5      0.0    1.0   0.000000  0.000000  0.000000  0.000000

In [75]:
ts_pivot = ts_pivot.reset_index()
ts_pivot

year month yearmonth   url           k1_freq             k2_freq  \
cluster                           1      2          1         2         1   
0        2022     1    2022.1  99.0  191.0   1.484848  0.853403  0.141414   
1        2022     2    2022.2  28.0   36.0   2.500000  1.222222  0.214286   
2        2022     3    2022.3  11.0   19.0   2.636364  1.578947  0.000000   
3        2022     4    2022.4   7.0    5.0   2.857143  0.000000  0.285714   
4        2022     5    2022.5   3.0    2.0   2.000000  0.000000  0.000000   
5        2022     6    2022.6  14.0    8.0   3.500000  0.750000  0.214286   
6        2022     7    2022.7   3.0    3.0   2.333333  2.000000  0.000000   
7        2022     8    2022.8   0.0    7.0   0.000000  1.428571  0.000000   
8        2022     9    2022.9   7.0    1.0   3.571429  0.000000  0.000000   
9        2022    10   2022.10   4.0    4.0   4.750000  0.000000  0.000000   
10       2022    11   2022.11   4.0    3.0   3.750000  2.333333  0.000000   
11       2022    12   2022.12   4.0    8.0   1.500000  2.250000  0.000000   
12       2023     1    2023.1   4.0   17.0   1.750000  1.470588  0.000000   
13       2023     2    2023.2   0.0    0.0   0.000000  0.000000  0.000000   
14       2023     3    2023.3   1.0    0.0  10.000000  0.000000  0.000000   
15       2023     4    2023.4   0.0    0.0   0.000000  0.000000  0.000000   
16       2023     5    2023.5   0.0    1.0   0.000000  0.000000  0.000000   

                   
cluster         2  
0        0.764398  
1        0.111111  
2        0.368421  
3        0.000000  
4        0.000000  
5        0.000000  
6        0.000000  
7        0.000000  
8        0.000000  
9        0.000000  
10       0.000000  
11       0.125000  
12       0.705882  
13       0.000000  
14       0.000000  
15       0.000000  
16       0.000000

In [76]:
pivot_column_names = ["year","month","yearmonth","c1_count","c2_count","c1_k1freq","c2_k1freq","c1_k2freq","c2_k2freq"]
ts_pivot.columns = pivot_column_names
ts_pivot

,year,month,yearmonth,c1_count,c2_count,c1_k1freq,c2_k1freq,c1_k2freq,c2_k2freq
0,2022,1,2022.1,99.0,191.0,1.484848,0.853403,0.141414,0.764398
1,2022,2,2022.2,28.0,36.0,2.500000,1.222222,0.214286,0.111111
2,2022,3,2022.3,11.0,19.0,2.636364,1.578947,0.000000,0.368421
3,2022,4,2022.4,7.0,5.0,2.857143,0.000000,0.285714,0.000000
4,2022,5,2022.5,3.0,2.0,2.000000,0.000000,0.000000,0.000000
5,2022,6,2022.6,14.0,8.0,3.500000,0.750000,0.214286,0.000000
6,2022,7,2022.7,3.0,3.0,2.333333,2.000000,0.000000,0.000000
7,2022,8,2022.8,0.0,7.0,0.000000,1.428571,0.000000,0.000000
8,2022,9,2022.9,7.0,1.0,3.571429,0.000000,0.000000,0.000000
9,2022,10,2022.10,4.0,4.0,4.750000,0.000000,0.000000,0.000000


# Export ts & ts_pivot

In [77]:
df_merge_filename = "article_kcounts_merged.xlsx"
df_merge.to_excel(path_analysis + df_merge_filename,index=False)

In [78]:
ts_filename = "ts.xlsx"
ts.to_excel(path_analysis + ts_filename,index=False)

In [79]:
ts_pivot_filename = "ts_pivot.xlsx"
ts_pivot.to_excel(path_analysis + ts_pivot_filename,index=False)